In [72]:
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
from tmdbv3api import Movie
from dotenv import load_dotenv
import os
import requests
from datetime import datetime

In [5]:
load_dotenv()

True

In [7]:
tmdb = TMDb()
tmdb.api_key = os.getenv('API_KEY')
tmdb.language = 'en'
tmdb.debug = True

In [97]:
movie = Movie()


In [17]:
search = movie.search('Chappaquiddick')
for res in search:
    url = f'https://api.themoviedb.org/3/movie/157336?api_key={os.getenv("API_KEY")}'

Chappaquiddick


- Movie Details =>  http://api.themoviedb.org/3/movie/{id}?api_key={api_key}
- Cast Details => https://api.themoviedb.org/3/movie/157336/credits?api_key={os.getenv("API_KEY")}



In [23]:
# Extracting names of all the movies from 2018
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_American_films_of_2018")

In [51]:
# In the data list, from index 2 - 5 movies are stored 
# Extracting the names of the movies
names = []

In [85]:
for i in range(2, 6):
    for i in data[i]['Title']:
        names.append(i)




In [95]:

movie_id = []
movie_not_found = []

- The TMDB API returns all the movies containing that word we only want movies from 2018 with that names

In [98]:
date_format = "%Y-%m-%d"
start_date = datetime.strptime("2018-01-01", date_format)
end_date = datetime.strptime("2018-12-31", date_format)
for name in names:
    search = movie.search(name)
    for res in search:
        if(res['release_date']):
            if (datetime.strptime(res['release_date'], date_format) >= start_date and datetime.strptime(res['release_date'], date_format) <= end_date):
                movie_id.append((name, res['id']))
                break      
        else :
            print("Does not exist")
    else :
        print("Removed " + name + " from the names list")
        movie_not_found.append(name)
        # names.remove('name')

   

Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist


In [130]:
details = []

In [131]:
def get_genres(obj):
    genres = []

    # print(type(obj))
    for i in obj:
        # print(i['name'])
        genres.append(i['name'])
    return genres

In [132]:
for i in movie_id:

    url = f'https://api.themoviedb.org/3/movie/{i[1]}?api_key={os.getenv("API_KEY")}'
    response = requests.get(url)
    data_json = response.json()
    movie_details = {}
    movie_details['name'] = data_json['title']
    movie_details['id'] = i[1]
    movie_details['overview'] = data_json['overview'] 
    movie_details['genres'] = get_genres(data_json['genres'])
    details.append(movie_details)


    


In [134]:
df = pd.DataFrame(details)

In [135]:
df

,name,id,overview,genres
0,Insidious: The Last Key,406563,Parapsychologist Elise Rainier and her team tr...,"[Horror, Mystery, Thriller]"
1,The Strange Ones,426258,Mysterious events surround the travels of two ...,"[Thriller, Drama]"
2,Sweet Country,468210,"In 1929, an Australian Aboriginal stockman kil...","[Western, Crime, Thriller, Drama]"
3,The Commuter,399035,"A businessman, on his daily commute home, gets...","[Action, Thriller, Mystery]"
4,Proud Mary,442064,Mary is a hit woman working for an organized c...,"[Thriller, Action, Crime]"
...,...,...,...,...
511,Holmes & Watson,426563,Detective Sherlock Holmes and Dr. John Watson ...,"[Mystery, Adventure, Comedy, Crime]"
512,Vice,429197,"George W. Bush picks Dick Cheney, the CEO of H...","[Comedy, Drama]"
513,On the Basis of Sex,339380,Young lawyer Ruth Bader Ginsburg teams with he...,"[Drama, History]"
514,Destroyer,471507,"When Erin Bell was a young cop, she was given ...","[Thriller, Crime, Drama, Action]"


In [150]:
def get_actors(id):
    url  =  f'https://api.themoviedb.org/3/movie/{id}/credits?api_key={os.getenv("API_KEY")}'
    resonse = requests.get(url)
    data_json = resonse.json()
    obj = data_json['cast']
    actors = []
    length = 5 if len(obj) > 5 else len(obj)
    for i in range(length):
        actors.append(obj[i]['name'])
    return actors

In [160]:
def get_directors(id):
    url  =  f'https://api.themoviedb.org/3/movie/{id}/credits?api_key={os.getenv("API_KEY")}'
    resonse = requests.get(url)
    data_json = resonse.json()
    obj = data_json['crew']
    directors = []
    for i in obj:
        if(i['job'] == 'Director'):
            directors.append(i['name'])
            break
    return directors

In [152]:
df['actors'] = df['id'].apply(get_actors)

In [153]:
df.head()

,name,id,overview,genres,actors
0,Insidious: The Last Key,406563,Parapsychologist Elise Rainier and her team tr...,"[Horror, Mystery, Thriller]","[Lin Shaye, Leigh Whannell, Angus Sampson, Kir..."
1,The Strange Ones,426258,Mysterious events surround the travels of two ...,"[Thriller, Drama]","[Alex Pettyfer, James Freedson-Jackson, Marin ..."
2,Sweet Country,468210,"In 1929, an Australian Aboriginal stockman kil...","[Western, Crime, Thriller, Drama]","[Hamilton Morris, Bryan Brown, Sam Neill, Thom..."
3,The Commuter,399035,"A businessman, on his daily commute home, gets...","[Action, Thriller, Mystery]","[Liam Neeson, Vera Farmiga, Patrick Wilson, Jo..."
4,Proud Mary,442064,Mary is a hit woman working for an organized c...,"[Thriller, Action, Crime]","[Taraji P. Henson, Jahi Di'Allo Winston, Danny..."


In [161]:
df['director'] = df['id'].apply(get_directors)

In [162]:
df.head()

,name,id,overview,genres,actors,director
0,Insidious: The Last Key,406563,Parapsychologist Elise Rainier and her team tr...,"[Horror, Mystery, Thriller]","[Lin Shaye, Leigh Whannell, Angus Sampson, Kir...",[Adam Robitel]
1,The Strange Ones,426258,Mysterious events surround the travels of two ...,"[Thriller, Drama]","[Alex Pettyfer, James Freedson-Jackson, Marin ...",[Lauren Wolkstein]
2,Sweet Country,468210,"In 1929, an Australian Aboriginal stockman kil...","[Western, Crime, Thriller, Drama]","[Hamilton Morris, Bryan Brown, Sam Neill, Thom...",[Warwick Thornton]
3,The Commuter,399035,"A businessman, on his daily commute home, gets...","[Action, Thriller, Mystery]","[Liam Neeson, Vera Farmiga, Patrick Wilson, Jo...",[Jaume Collet-Serra]
4,Proud Mary,442064,Mary is a hit woman working for an organized c...,"[Thriller, Action, Crime]","[Taraji P. Henson, Jahi Di'Allo Winston, Danny...",[Babak Najafi]


In [163]:
def remove_space(obj):
    l = []
    for i in obj:
        l.append(i.replace(" ", ""))
    return l

In [165]:
df['actors'] = df['actors'].apply(remove_space)

In [168]:
df['overview'] = df['overview'].apply(lambda x : x.split())

In [170]:
df['director'] = df['director'].apply(remove_space)

In [172]:
df['tags'] = df['overview'] + df['genres'] + df['actors'] + df['director']

In [174]:
df['tags'] = df['tags'].apply(lambda x : " ".join(x))

In [175]:
df.head()

,name,id,overview,genres,actors,director,tags
0,Insidious: The Last Key,406563,"[Parapsychologist, Elise, Rainier, and, her, t...","[Horror, Mystery, Thriller]","[LinShaye, LeighWhannell, AngusSampson, KirkAc...",[AdamRobitel],Parapsychologist Elise Rainier and her team tr...
1,The Strange Ones,426258,"[Mysterious, events, surround, the, travels, o...","[Thriller, Drama]","[AlexPettyfer, JamesFreedson-Jackson, MarinIre...",[LaurenWolkstein],Mysterious events surround the travels of two ...
2,Sweet Country,468210,"[In, 1929,, an, Australian, Aboriginal, stockm...","[Western, Crime, Thriller, Drama]","[HamiltonMorris, BryanBrown, SamNeill, ThomasM...",[WarwickThornton],"In 1929, an Australian Aboriginal stockman kil..."
3,The Commuter,399035,"[A, businessman,, on, his, daily, commute, hom...","[Action, Thriller, Mystery]","[LiamNeeson, VeraFarmiga, PatrickWilson, Jonat...",[JaumeCollet-Serra],"A businessman, on his daily commute home, gets..."
4,Proud Mary,442064,"[Mary, is, a, hit, woman, working, for, an, or...","[Thriller, Action, Crime]","[TarajiP.Henson, JahiDi'AlloWinston, DannyGlov...",[BabakNajafi],Mary is a hit woman working for an organized c...


In [176]:
new_df = df[['id', 'name', 'tags']]

In [177]:
new_df

,id,name,tags
0,406563,Insidious: The Last Key,Parapsychologist Elise Rainier and her team tr...
1,426258,The Strange Ones,Mysterious events surround the travels of two ...
2,468210,Sweet Country,"In 1929, an Australian Aboriginal stockman kil..."
3,399035,The Commuter,"A businessman, on his daily commute home, gets..."
4,442064,Proud Mary,Mary is a hit woman working for an organized c...
...,...,...,...
511,426563,Holmes & Watson,Detective Sherlock Holmes and Dr. John Watson ...
512,429197,Vice,"George W. Bush picks Dick Cheney, the CEO of H..."
513,339380,On the Basis of Sex,Young lawyer Ruth Bader Ginsburg teams with he...
514,471507,Destroyer,"When Erin Bell was a young cop, she was given ..."


In [178]:

df = new_df.rename(columns = {'id' : 'movie_id', 'name' : 'title'})

In [181]:
df.to_csv('data_2018.csv', index = False)